In [1]:
import pandas as pd
import json
import mapply
import re
pd.set_option('display.max_colwidth', None)  # Set maximum width to terminal width
pd.set_option('display.expand_frame_repr', True)  # Expand DataFrame representation
mapply.init(
    n_workers=-1,
    chunk_size=1000,
    max_chunks_per_worker=0,
    progressbar=True
)

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# Set the path to your large JSON file
json_file_path = "/home/user/ItemDataQuality/itemdataqualityrepo/ingrednets.csv"

# Define the cleaned_data size (adjust as needed)
chunk_size = 10000

# Initialize an empty list to store cleaned DataFrames
cleaned_dataframes = []

# Iterate over chunks
for chunk in pd.read_csv(json_file_path,delimiter="\t" ,chunksize=chunk_size):
    # Append the cleaned chunk to the list
    cleaned_dataframes.append(chunk)

# Concatenate all cleaned chunks into a single DataFrame
cleaned_data = pd.concat(cleaned_dataframes, ignore_index=True)



/tmp/ipykernel_1685/1388434961.py:11: DtypeWarning: Columns (72) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(json_file_path,delimiter="\t" ,chunksize=chunk_size):
/tmp/ipykernel_1685/1388434961.py:11: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(json_file_path,delimiter="\t" ,chunksize=chunk_size):
/tmp/ipykernel_1685/1388434961.py:11: DtypeWarning: Columns (0,11,33,35,72) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(json_file_path,delimiter="\t" ,chunksize=chunk_size):
/tmp/ipykernel_1685/1388434961.py:11: DtypeWarning: Columns (33,35) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(json_file_path,delimiter="\t" ,chunksize=chunk_size):
/tmp/ipykernel_1685/1388434961.py:11: DtypeWarning: Columns (0) have mixed types. Specify dtype option on 

In [13]:
cleaned_data['glycemic-index_100g'].value_counts()


glycemic-index_100g
54.0    1
30.0    1
48.7    1
70.0    1
44.0    1
39.0    1
14.0    1
Name: count, dtype: int64

In [5]:
for chunk in df:
    # Process the first chunk (100 records)
    print(chunk.head())  # View the first 5 records
    break 

ParserError: Error tokenizing data. C error: Expected 59 fields in line 4, saw 71


In [4]:
  
cleaned_data = cleaned_data.drop(['category','tech1','fit','tech2','similar_item','date','also_view','also_buy'],axis=1)


In [5]:
cleaned_data = cleaned_data.applymap(lambda x: '; '.join(map(str, x)) if x is not None and isinstance(x, list) else x)


/tmp/ipykernel_6384/4085774542.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  cleaned_data = cleaned_data.applymap(lambda x: '; '.join(map(str, x)) if x is not None and isinstance(x, list) else x)


In [6]:
cleaned_data['rank'] = cleaned_data['rank'].mapply(lambda x: re.sub('[^a-zA-Z0-9&, ]|amp', '', x) if x is not None and isinstance(x, str) else x)


/home/user/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 786/786 [00:47<00:00, 16.54it/s]


In [7]:
normalized_data = pd.json_normalize(cleaned_data['details'])
normalized_data.columns

Index(['ASIN: ', 'Item model number:', 'Paperback:', 'Publisher:', 'Language:',
       'ISBN-10:', 'ISBN-13:', '\n    Product Dimensions: \n    ',
       'Shipping Weight:', 'Electronics:', '\n    Package Dimensions: \n    ',
       'Hardcover:', 'ASIN:', ' Date first listed on Amazon:', 'Series:',
       'Domestic Shipping: ', 'International Shipping: ', 'UPC:',
       'Discontinued by manufacturer:', 'Batteries',
       '\n    Item Weight: \n    ', 'Shipping Advisory:', 'Shipping: ',
       'Perfect Paperback:', ' UNSPSC Code:', 'Downloading:', 'Note:',
       'Cards:'],
      dtype='object')

In [ ]:
#Assign None if below three columns not present in normalized data
normalized_data['product_dimensions']=None
normalized_data['item_weight']=None
normalized_data['shipping_weight']=None

In [ ]:
# Add the normalized data back to the original DataFrame based on the index
cleaned_data = pd.concat([cleaned_data, normalized_data], axis=1, join='inner')


In [ ]:
cleaned_data['category']='Electronics'


In [ ]:
cleaned_data.columns = (cleaned_data.columns
                        .str.strip()  # Remove leading/trailing spaces
                        .str.lower()  # Convert to lowercase (optional for consistency)
                        .str.replace(':', '') # Remove colons characters
                        .str.replace('\n', '') # Remove newline characters
                        .str.replace('.', '_') # Replace periods with underscores
                        .str.replace(' ', '_'))  # Replace spaces with underscores

In [ ]:
#drop duplicate columns
cleaned_data = cleaned_data.loc[:, ~cleaned_data.columns.duplicated()]

In [ ]:
cleaned_data = cleaned_data[['asin','category','main_cat','title','description', 
        'brand','price','product_dimensions','item_weight', 'shipping_weight',
        'imageurl', 'imageurlhighres','feature','rank']]

In [ ]:
cleaned_data.to_parquet('gs://data_quality_final/parquet_data_files/meta_Electronics',index=False)